# MÔ HÌNH PHÂN LOẠI ÂM THANH DỰA TRÊN MÔ HÌNH CNN14

In [1]:
!pip install paddlepaddle
import os
import librosa
import numpy as np
import cv2
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader, random_split


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00


1. Chuẩn hóa theo dải giá trị
Đoạn code này giả định rằng sample là một tensor (hoặc mảng numpy) chứa giá trị của một ảnh âm thanh sau khi được xử lý thành hình ảnh (ví dụ: mel spectrogram) và có giá trị nằm trong khoảng từ 0 đến 255 (hoặc bất kỳ dải giá trị nào tương tự).
**sample = sample / 255.0**
Bước này chia toàn bộ giá trị của sample cho 255.0. Điều này sẽ đưa các giá trị từ dải [0, 255] về dải [0.0, 1.0]. Đây là bước chuẩn hóa đơn giản để đưa dữ liệu về cùng một phạm vi.
2. Đưa về trung tâm 0
**sample -= 0.5**
Bước này trừ đi 0.5 từ mỗi giá trị trong sample. Sau bước này, các giá trị trong sample sẽ nằm trong khoảng [-0.5, 0.5]. Điều này giúp dịch chuyển dữ liệu để tâm tại 0, giúp mô hình dễ dàng học các biến đổi tuyến tính.
3. Scale lại để có dải giá trị rộng hơn
sample *= 2.0
Bước cuối cùng nhân mỗi giá trị trong sample với 2.0. Kết quả là các giá trị trong sample sẽ nằm trong khoảng [-1.0, 1.0]. Điều này làm mở rộng khoảng giá trị của dữ liệu, giúp cho mô hình dễ dàng học hơn và có thể cải thiện độ chính xác của mô hình.

In [2]:
class Normalize(object):
    def __call__(self, sample):
        sample = sample / 255.0
        sample -= 0.5
        sample *= 2.0
        return sample


Lớp AudioDataset dành cho việc xử lý dữ liệu âm thanh. Hãy đi vào từng phương thức và thuộc tính của lớp này để hiểu rõ hơn:
1. Phương thức __init__
audio_dir: Đường dẫn đến thư mục chứa dữ liệu âm thanh.
labels: Danh sách các nhãn (labels) cho các lớp âm thanh (vd: ['Bac', 'Nam', 'Trung']).
transform: Hàm biến đổi để xử lý ảnh (nếu có).
file_paths: Danh sách các đường dẫn tới các file âm thanh (.wav) trong thư mục audio_dir.
targets: Danh sách các chỉ số tương ứng với nhãn của mỗi file âm thanh trong file_paths.
Trong phương thức __init__, một lần khởi tạo đối tượng AudioDataset, nó sẽ duyệt qua từng nhãn và từng file âm thanh trong thư mục audio_dir. Nếu file đó là file âm thanh (.wav), nó sẽ thêm đường dẫn của file và chỉ số của nhãn vào file_paths và targets tương ứng.
2. Phương thức __len__
Phương thức này trả về số lượng mẫu trong dataset, tức là số lượng file âm thanh đã được tải và chuẩn bị trong file_paths.
3. Phương thức __getitem__
idx: Chỉ số của mẫu trong dataset mà bạn muốn truy cập.
Phương thức __getitem__ được gọi khi bạn gọi dataset[idx] để lấy một mẫu từ dataset.

file_path: Lấy đường dẫn của file âm thanh tại vị trí idx trong file_paths.
target: Lấy nhãn của file âm thanh tại vị trí idx trong targets.
Tiếp theo, nó sẽ:

Sử dụng librosa để tải âm thanh từ file_path, giới hạn độ dài là 2.5 giây, bắt đầu từ vị trí 0.6 giây.
Tính toán Mel spectrogram và chuyển đổi sang đơn vị độ dB (mel_spect_db).
Thực hiện resize ảnh đến kích thước 128x128 sử dụng OpenCV (cv2.resize).
Mở rộng chiều của ảnh để có một kênh duy nhất (single channel) và chuyển đổi sang kiểu dữ liệu float32.
Cuối cùng, nếu transform được cung cấp (thường là một hàm biến đổi), nó sẽ áp dụng transform lên ảnh (vd: chuẩn hóa, augmentation,...).

Kết quả trả về là cặp (img, target), trong đó img là đầu vào cho mô hình (ảnh Mel spectrogram đã được xử lý), và target là nhãn tương ứng của mẫu đó.



In [3]:
class AudioDataset(Dataset):
    def __init__(self, audio_dir, labels, transform=None):
        self.audio_dir = audio_dir
        self.labels = labels
        self.transform = transform
        self.file_paths = []
        self.targets = []

        for label in labels:
            folder_path = os.path.join(audio_dir, label)
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.wav'):
                    self.file_paths.append(os.path.join(folder_path, file_name))
                    self.targets.append(labels.index(label))

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        target = self.targets[idx]

        y, sr = librosa.load(file_path, duration=2.5, offset=0.6)
        mel_spect = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spect_db = librosa.power_to_db(mel_spect, ref=np.max)
        img = cv2.resize(mel_spect_db, (128, 128))
        img = np.expand_dims(img, axis=0).astype(np.float32)  # Convert to single channel

        if self.transform:
            img = self.transform(img)

        return img, target


1. Định nghĩa nhãn và tải dữ liệu
labels: Danh sách các nhãn của dữ liệu âm thanh. Trong trường hợp này, có ba nhãn là 'Bac', 'Nam', và 'Trung'.
audio_dir: Đường dẫn tới thư mục chứa dữ liệu âm thanh. Trong ví dụ này, dữ liệu âm thanh được lưu tại '/content/drive/MyDrive/VOICE/dulieu'.
dataset: Đối tượng của lớp AudioDataset, được khởi tạo với audio_dir, labels, và một đối tượng biến đổi Normalize() để chuẩn hóa dữ liệu.

2. Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_size: Số lượng mẫu trong tập huấn luyện, lấy là 80% của số lượng mẫu trong dataset.
test_size: Số lượng mẫu trong tập kiểm tra, bằng phần còn lại của dataset sau khi lấy tập huấn luyện.
random_split(dataset, [train_size, test_size]): Hàm random_split từ paddle.io được sử dụng để chia dataset thành hai phần tập huấn luyện (train_dataset) và tập kiểm tra (test_dataset).

3. Tạo DataLoader
train_loader: DataLoader cho tập huấn luyện, được tạo từ train_dataset, với batch_size là 32 (tức là mỗi lần đọc vào mô hình sẽ được cung cấp 32 mẫu).
test_loader: DataLoader cho tập kiểm tra, được tạo từ test_dataset, cũng với batch_size là 32. shuffle=False để đảm bảo dữ liệu kiểm tra không bị xáo trộn.


In [4]:
# Define Labels and Load Dataset
labels = ['Bac', 'Nam', 'Trung']
audio_dir = '/content/drive/MyDrive/VOICE/dulieu'
dataset = AudioDataset(audio_dir, labels, transform=Normalize())

# Split Dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Lớp ConvBlock trong đoạn code là một khối convolution được xây dựng để sử dụng trong mô hình của bạn. Hãy đi vào từng thành phần của nó để hiểu cách hoạt động:

1. Hàm khởi tạo __init__
in_channels: Số kênh đầu vào của dữ liệu.
out_channels: Số kênh đầu ra (số lượng filter) của lớp convolution.
Trong hàm khởi tạo này:

self.conv1 và self.conv2 là hai lớp convolution 2D với kernel size là 3x3, stride là 1 và padding là 1. in_channels là số kênh đầu vào của lớp convolution đầu tiên và out_channels là số kênh đầu ra của cả hai lớp convolution.
self.bn1 và self.bn2 là hai lớp Batch Normalization để chuẩn hóa giá trị đầu ra của các lớp convolution trước khi đưa vào hàm kích hoạt (ReLU).

2. Phương thức forward:
- x: Đầu vào của khối convolution, là một tensor.
pool_size: Kích thước cửa sổ pooling.
pool_type: Loại pooling được sử dụng ('max', 'avg', 'avg+max').
- Trong phương thức forward:

+ Đầu tiên, đầu vào x được đi qua lớp convolution đầu tiên (self.conv1), sau đó chuẩn hóa bằng Batch Normalization (self.bn1) và áp dụng hàm kích hoạt ReLU (F.relu).
+ Tiếp theo, đầu ra của lớp convolution thứ nhất được đi qua lớp convolution thứ hai (self.conv2), sau đó chuẩn hóa bằng Batch Normalization (self.bn2) và áp dụng hàm kích hoạt ReLU.
+ Sau khi hoàn thành các phép tính trên, lựa chọn loại pooling dựa trên tham số pool_type:
 + Nếu pool_type là 'max', sử dụng pooling max (F.max_pool2d).
 + Nếu pool_type là 'avg', sử dụng pooling average (F.avg_pool2d).
 + Nếu pool_type là 'avg+max', thực hiện cả hai loại pooling và cộng kết quả lại với nhau.
 + Nếu không phù hợp với bất kỳ loại nào trong các loại trên, raise một ngoại lệ (Exception).
+ Kết quả của phương thức forward là đầu ra của khối convolution sau khi áp dụng các lớp convolution, Batch Normalization và pooling tương ứng. Đây là một khối cơ bản được sử dụng để xây dựng mô hình CNN trong bài toán phân loại âm thanh của bạn.


In [5]:
class ConvBlock(nn.Layer):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2D(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2D(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2D(out_channels)
        self.bn2 = nn.BatchNorm2D(out_channels)

    def forward(self, x, pool_size=(2, 2), pool_type='avg'):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        return x


Lớp Cnn14 là một mô hình mạng nơ-ron tích chập (CNN) được thiết kế để phân loại âm thanh vào ba lớp ("Bac", "Nam", "Trung"). Hãy đi vào từng thành phần của lớp này để hiểu cách hoạt động:
1. Hàm khởi tạo __init__:
- self.bn0: Lớp Batch Normalization cho đầu vào đầu tiên, có 1 kênh.
- self.conv_block1 đến self.conv_block6: Sáu khối convolution, mỗi khối được xây dựng từ lớp ConvBlock. Các khối này có số kênh đầu vào và đầu ra tăng dần để học các đặc trưng phức tạp hơn khi đi sâu vào mạng.
- self.fc1: Lớp fully connected (dense) với đầu vào 2048 và đầu ra 2048.
- self.fc_audioset: Lớp fully connected cuối cùng với đầu vào 2048 và đầu ra 3, tương ứng với số lớp đầu ra (Ba, Nam, Trung).

2. Phương thức forward:
- x: Đầu vào của mô hình, là một tensor.
- self.bn0(x): Chuẩn hóa đầu vào đầu tiên bằng Batch Normalization.
- self.conv_block1 đến self.conv_block6: Các khối convolution tuần tự, mỗi khối là một đối tượng của lớp ConvBlock được gọi với tham số pool_size=(2, 2) và pool_type='avg'. Đây là các khối tích chập liên tiếp nhau để học các đặc trưng từ dữ liệu.
- F.dropout(x, p=0.2, training=self.training): Áp dụng dropout để ngăn chặn overfitting trong quá trình huấn luyện. p=0.2 là tỷ lệ dropout (20%).
- x.mean(axis=3, keepdim=True): Lấy giá trị trung bình theo chiều thứ 3 của x.
- x1 = x.max(axis=2, keepdim=True): Lấy giá trị lớn nhất theo chiều thứ 2 của x.
- x2 = x1.mean(axis=2, keepdim=True): Lấy giá trị trung bình theo chiều thứ 2 của x1.
- x = x1 + x2: Cộng hai tensor x1 và x2.
- x.squeeze(): Loại bỏ các chiều có kích thước là 1.
- F.dropout(x, p=0.5, training=self.training): Áp dụng dropout với tỷ lệ 50% trước khi đi qua lớp fully connected đầu tiên (self.fc1).
- F.relu(self.fc1(x)): Áp dụng hàm kích hoạt ReLU cho đầu ra của lớp fully connected đầu tiên.

In [6]:
class Cnn14(nn.Layer):
    def __init__(self):
        super(Cnn14, self).__init__()
        self.bn0 = nn.BatchNorm2D(1)
        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)
        self.fc1 = nn.Linear(2048, 2048)
        self.fc_audioset = nn.Linear(2048, 3)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = x.mean(axis=3, keepdim=True)
        x1 = x.max(axis=2, keepdim=True)
        x2 = x1.mean(axis=2, keepdim=True)
        x = x1 + x2
        x = x.squeeze()
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc1(x))
        return x


Lớp ESCModel là một mô hình tổng hợp dựa trên mô hình CNN Cnn14 để phân loại âm thanh vào ba lớp ("Bac", "Nam", "Trung").
1. Hàm khởi tạo __init__
- self.audioset_model: Lớp Cnn14 được khởi tạo làm thành phần của mô hình. Đây là mô hình CNN để trích xuất đặc trưng từ dữ liệu âm thanh.
- self.fc_esc50: Lớp fully connected cuối cùng với đầu vào 2048 (đầu ra của Cnn14) và đầu ra 3, tương ứng với số lớp đầu ra ("Bac", "Nam", "Trung").

2. Phương thức forward
- x: Đầu vào của mô hình, là một tensor.
self.audioset_model(x): Đưa đầu vào x qua mô hình Cnn14 để trích xuất đặc trưng.
- F.dropout(out, p=0.5, training=self.training): Áp dụng dropout với tỷ lệ 50% cho đầu ra của Cnn14, nhằm ngăn chặn overfitting trong quá trình huấn luyện.
- self.fc_esc50(out): Đưa đầu ra của dropout qua lớp fully connected cuối cùng để tính toán logits (điểm số) cho các lớp đầu ra.

In [7]:
class ESCModel(paddle.nn.Layer):
    def __init__(self):
        super(ESCModel, self).__init__()
        self.audioset_model = Cnn14()
        self.fc_esc50 = paddle.nn.Linear(2048, 3)

    def forward(self, x):
        out = self.audioset_model(x)
        out = F.dropout(out, p=0.5, training=self.training)
        logits = self.fc_esc50(out)
        return logits


QUÁ TRÌNH HUẤN LUYỆN:
1. Khởi tạo mô hình và các thành phần cần thiết
- Khởi tạo một đối tượng ESCModel, mô hình sẽ được sử dụng để huấn luyện và dự đoán.

2. Định nghĩa hàm loss và optimizer
- CrossEntropyLoss: Hàm mất mát dùng để tính toán sự mất mát giữa các dự đoán và nhãn.
- Adam: Trình tối ưu hóa Adam được sử dụng để cập nhật các tham số của mô hình dựa trên gradient của hàm mất mát.

3. Huấn luyện mô hình
- Vòng lặp for epoch in range(epochs) chạy qua số lượng epochs đã chỉ định.
- model.train(): Thiết lập mô hình ở chế độ huấn luyện.
- train_loader: Dataloader chứa dữ liệu huấn luyện.
- Vòng lặp bên trong lặp qua train_loader để lấy dữ liệu đầu vào (inputs) và nhãn (labels).
- outputs = model(inputs): Dự đoán đầu ra của mô hình.
- loss = criterion(outputs, labels): Tính toán hàm mất mát giữa dự đoán và nhãn.
- loss.backward(): Lan truyền ngược để tính gradient của các tham số.
-optimizer.step(): Cập nhật các tham số của mô hình bằng cách sử dụng optimizer.
- optimizer.clear_grad(): Xóa gradient sau mỗi lần cập nhật tham số để chuẩn bị cho lần cập nhật tiếp theo.
- running_loss += loss.item(): Cộng dồn hàm mất mát để tính tổng hàm mất mát trung bình sau mỗi epoch.
- print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}'): In ra giá trị trung bình của hàm mất mát trong quá trình huấn luyện.

4. Đánh giá mô hình
5. Lưu mô hình

In [8]:
model = ESCModel()

criterion = paddle.nn.CrossEntropyLoss()
optimizer = paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with paddle.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = paddle.topk(outputs, k=1)
            total += labels.shape[0]
            correct += (predicted.squeeze() == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

# Save model
paddle.save(model.state_dict(), 'model.pdparams')


/usr/local/lib/python3.10/dist-packages/paddle/nn/layer/norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


Epoch 1/10, Loss: 3.272491453862232
Validation Accuracy: 89.85507246376811%
Epoch 2/10, Loss: 1.5790609271886449
Validation Accuracy: 89.85507246376811%
Epoch 3/10, Loss: 0.9462968382156558
Validation Accuracy: 88.40579710144928%
Epoch 4/10, Loss: 0.6088133835130267
Validation Accuracy: 89.85507246376811%
Epoch 5/10, Loss: 0.8691312821562557
Validation Accuracy: 89.85507246376811%
Epoch 6/10, Loss: 0.6693166063891517
Validation Accuracy: 89.85507246376811%
Epoch 7/10, Loss: 1.6971213484389915
Validation Accuracy: 89.85507246376811%
Epoch 8/10, Loss: 0.5539501698480712
Validation Accuracy: 89.85507246376811%
Epoch 9/10, Loss: 0.5426625675625272
Validation Accuracy: 86.23188405797102%
Epoch 10/10, Loss: 0.746419271454215
Validation Accuracy: 89.85507246376811%
